Tags scraped from goodreads are user-provided and therefore rather messy. To reduce the huge number of rarely used tags we reduce the original dataset by only keeping tags which were used at least 100 times.

In [1]:
%%bash
make data-prep

cd ./goodbooks-10k && \
xsv join tag_id tags.csv tag_id book_tags.csv | xsv search -s count "\d{3,}" | xsv select tag_id,goodreads_book_id,count > book_tags_reduced_imm.csv  && \
xsv join goodreads_book_id book_tags_reduced_imm.csv goodreads_book_id books.csv |  xsv select tag_id,book_id,goodreads_book_id,count > book_tags_reduced.csv && \
xsv join tag_id tags.csv tag_id book_tags.csv | xsv search -s count "\d{3,}" | xsv select tag_id,tag_name | tail -n +2 | sort | uniq | sort | cat <(echo "tag_id,tag_name") - | xsv sort --select tag_id --numeric > tags_reduced.csv && \
rm book_tags_reduced_imm.csv


We read in the resulting dataset:

In [7]:
# load CSV files with pandas
import pandas as pd

from pathlib import Path

cwd = Path().absolute()
data_path = cwd / "goodbooks-10k"

books_csv = pd.read_csv(
    data_path / "books.csv",
    index_col = ["book_id"],
    dtype={
        "isbn": str,
        "isbn13": float,  # original dataset encoded these erronous as floats
        "original_publication_year": float,
        "original_title": str,
        "title": str,
        "language_code": str,
        "average_rating": float,
        "ratings_count": int,
    },
)
books_csv["original_publication_year"] = books_csv["original_publication_year"].astype("Int32")
books_csv["isbn13"] = books_csv["isbn13"].astype("Int64")
books_csv

,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
book_id,,,,,,,,,,,,,,,,,,,,,
1,2767052,2767052,2792775,272,439023483,9780439023480,Suzanne Collins,2008,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
2,3,3,4640799,491,439554934,9780439554930,"J.K. Rowling, Mary GrandPré",1997,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
3,41865,41865,3212258,226,316015849,9780316015840,Stephenie Meyer,2005,Twilight,"Twilight (Twilight, #1)",...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
4,2657,2657,3275794,487,61120081,9780061120080,Harper Lee,1960,To Kill a Mockingbird,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
5,4671,4671,245494,1356,743273567,9780743273560,F. Scott Fitzgerald,1925,The Great Gatsby,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,7130616,7130616,7392860,19,441019455,9780441019460,Ilona Andrews,2010,Bayou Moon,"Bayou Moon (The Edge, #2)",...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9997,208324,208324,1084709,19,067973371X,9780679733710,Robert A. Caro,1990,Means of Ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)",...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9998,77431,77431,2393986,60,039330762X,9780393307630,Patrick O'Brian,1977,The Mauritius Command,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...


In [8]:
books_csv.iloc[[74]][["original_title","title", "isbn", "isbn13"]]

,original_title,title,isbn,isbn13
book_id,,,,
75,NaN,"Bridget Jones's Diary (Bridget Jones, #1)",014028009X,9780140280100


The ISBN13 numbers are encoded as floats, we have to parse them into the canonical ISBN from of "978-01-40-280100".
Be aware that there are some missing values in this column.

In [3]:
books_csv['isbn13'] = books_csv['isbn13'].astype(str)

books_csv['isbn13'] = books_csv['isbn13'].str.replace('.0', '')
books_csv['isbn13'] = books_csv['isbn13'].str.zfill(13)
books_csv['isbn13'] = books_csv['isbn13'].str[:3] + '-' + books_csv['isbn13'].str[3:5] + '-' + books_csv['isbn13'].str[5:7] + '-' + books_csv['isbn13'].str[7:13]
books_csv['isbn13'] = books_csv['isbn13'].where(~books_csv['isbn13'].str.contains('<NA>'), 'not available')

The ISBN numbers are encoded as strings, we have to parse them into the canonical ISBN from of "0-14-02800-9X".
Be aware that there are some missing values in this column.

In [4]:
books_csv['isbn'] = books_csv['isbn'].astype(str)

books_csv['isbn'] = books_csv['isbn'].str.zfill(10)
books_csv['isbn'] = books_csv['isbn'].str[:1] + '-' + books_csv['isbn'].str[1:3] + '-' + books_csv['isbn'].str[3:8] + '-' + books_csv['isbn'].str[8:10]
books_csv['isbn'] = books_csv['isbn'].where(~books_csv['isbn'].str.contains('n-an'), 'not available')

If one book is missing its original title, we substitute the value of the title field, but without any series information which might be appendend in paranthesis:

```
title("Bridget Jones's Diary (Bridget Jones, #1") -> original_title("Bridget Jones's Diary")
```

In [5]:
books_csv['original_title'] = books_csv['original_title'].fillna(books_csv['title']).str.replace(r'\s*[(].*[)]\s*$', '', regex=True)

In [6]:
books_csv.iloc[[105]][["original_title", "title", "isbn", "isbn13"]]

,original_title,title,isbn,isbn13
book_id,,,,
106,Bossypants,Bossypants,not available,not available


In [111]:
books_csv.iloc[[74]][["original_title","title", "isbn", "isbn13"]]

,original_title,title,isbn,isbn13
book_id,,,,
75,Bridget Jones's Diary,"Bridget Jones's Diary (Bridget Jones, #1)",0-14-02800-9X,978-01-40-280100


## Use dbpedia to extract additional information

In [43]:
books_csv["dbpedia_resource"] = ""
books_csv["genres"] = ""

Unfortunatly our dataset usually uses the ISBN of the best-selling edition of a book, while DBpedia seems to usually use the ISBN of a book's first edition.
Not a single ISBN within our dataset can be matched to a DBpedia entry:

In [ ]:
# Use SPARQL to find dbpedia entries
from SPARQLWrapper import SPARQLWrapper, CSV

import io
import string
import time


sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(CSV)

query = string.Template("""
select distinct ?book ?isbn (GROUP_CONCAT(DISTINCT ?genreLabel; SEPARATOR=",")) as ?genreLabels where {
  ?book a dbo:Book;
        dbo:isbn ?isbn;
        dbo:literaryGenre ?genre .
  ?genre rdfs:label ?genreLabel
  filter STRSTARTS(?isbn,"$isbn")
  filter langMatches(lang(?genreLabel),'en')
}
""")

matches_found=0
for index, row in books_csv.iterrows():
    sparql.setQuery(query.substitute({"isbn": row["isbn"]}))
    results = sparql.query().convert()
    results = pd.read_csv(io.BytesIO(results))

    if len(results) == 0:
        # no match
        continue
    
    matches_found += 1
    print(f"Found ISBN match for {row['original_title']}")
    if len(results) > 1:
        print(f"book {index} returned multiple results!")
    books_csv.at[index, "dbpedia_resource"] = results.iloc[0]["book"]
    books_csv.at[index, "genres"] = results.iloc[0]["genreLabels"]
    
    time.sleep(0.010) # sleep 10 milliseconds

print(f"{matches_found=}")

We have more luck by matching the book titles against DBpedia.
To avoid accumulating faulty data in our dataset we only enrich our dataset with DBpedia data if there is a single match for our book title.
If a query returns multiple matches we ignore its results.

In [84]:
# Use SPARQL to find dbpedia entries
from SPARQLWrapper import SPARQLWrapper, CSV
from tqdm.auto import tqdm

import io
import string
import time



sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(CSV)

query = string.Template("""
select distinct ?book ?label (GROUP_CONCAT(DISTINCT ?genreLabel; SEPARATOR=",")) as ?genreLabels where {
  ?book a dbo:Book;
        rdfs:label ?label;
        dbo:literaryGenre ?genre .
  ?genre rdfs:label ?genreLabel
  filter STRSTARTS(?label,"$title")
  filter langMatches(lang(?label),'en')
  filter langMatches(lang(?genreLabel),'en')
}
""")


matches_found=0
multiple_matches_indices = []
for index, row in (pbar := tqdm(books_csv.iterrows(), total=books_csv.shape[0])):
    pbar.set_description(f"{matches_found=}, returned-multiple={len(multiple_matches_indices)}")
    try:
        sparql.setQuery(query.substitute({"title": row["original_title"].replace('"', r'\"')}))
        results = sparql.query().convert()
        results = pd.read_csv(io.BytesIO(results))
    except Exception as exc:
        print(f"Error {index=}: {exc}")
        time.sleep(0.010) # sleep 10 milliseconds
        continue

    if len(results) == 0:
        # no match
        time.sleep(0.010) # sleep 10 milliseconds
        continue
    
    if len(results) > 1:
        multiple_matches_indices.append(index)
        time.sleep(0.010) # sleep 10 milliseconds
        continue

    matches_found += 1    
    books_csv.at[index, "dbpedia_resource"] = results.iloc[0]["book"]
    books_csv.at[index, "genres"] = results.iloc[0]["genreLabels"]
    
    time.sleep(0.010) # sleep 10 milliseconds

print(f"{matches_found=}")
print(f"{len(multiple_matches_indices)} book returned multiple matches")

matches_found=2919, returned-multiple=660:  98%|█████████▊| 9846/10000 [1:23:12<04:03,  1.58s/it]  

Error index=9846: HTTP Error 503: Service Temporarily Unavailable


matches_found=2955, returned-multiple=667: 100%|██████████| 10000/10000 [1:25:02<00:00,  1.96it/s]

matches_found=2955
667 book returned multiple matches


In [107]:
books_csv[["original_title", "dbpedia_resource", "genres"]]

,original_title,dbpedia_resource,genres
book_id,,,
1,The Hunger Games,http://dbpedia.org/resource/The_Hunger_Games_(...,"Adventure novel,Science fiction"
2,Harry Potter and the Philosopher's Stone,http://dbpedia.org/resource/Harry_Potter_and_t...,Fantasy novel
3,Twilight,,
4,To Kill a Mockingbird,http://dbpedia.org/resource/To_Kill_a_Mockingbird,"Bildungsroman,Southern Gothic"
5,The Great Gatsby,http://dbpedia.org/resource/The_Great_Gatsby,Tragedy
...,...,...,...
9996,Bayou Moon,,
9997,Means of Ascent,,
9998,The Mauritius Command,http://dbpedia.org/resource/The_Mauritius_Command,Historical novel


## Deduce book series from original title


In [20]:
with pd.option_context('display.max_colwidth', 200):
    display(books_csv.iloc[[1]][["original_title", "title"]])

,original_title,title
book_id,,
2,Harry Potter and the Philosopher's Stone,"Harry Potter and the Sorcerer's Stone (Harry Potter, #1)"


In [89]:
# Regex pattern
pattern = r"(.*)([(])(.*)(\s*#\s*\d*)"
# Extracting capture group three
books_csv['book_series'] = books_csv['title'].str.extract(pattern)[2]

In [106]:
books_csv['book_series'] = books_csv['book_series'].fillna('')
books_csv['book_series'] = books_csv['book_series'].str.strip()
books_csv['book_series'] = books_csv['book_series'].str.rstrip(',')
books_csv[["original_title", "book_series"]]

,original_title,book_series
book_id,,
1,The Hunger Games,The Hunger Games
2,Harry Potter and the Philosopher's Stone,Harry Potter
3,Twilight,Twilight
4,To Kill a Mockingbird,
5,The Great Gatsby,
...,...,...
9996,Bayou Moon,The Edge
9997,Means of Ascent,The Years of Lyndon Johnson
9998,The Mauritius Command,


In [96]:
books_csv.to_csv(data_path / "books_enriched.csv")

In [102]:
books_csv["book_series"].value_counts()

book_series
                     5561
In Death               41
Stephanie Plum         28
Hercule Poirot         25
Kinsey Millhone        24
                     ... 
Sarah Agnes Prine       1
InuYasha                1
Paddington              1
On the Island           1
Runaway Train           1
Name: count, Length: 1795, dtype: int64